<a href="https://colab.research.google.com/github/AsmaMora/Customer_Analytics/blob/main/MADT8101_Voice_of_customer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --pre pythainlp
!pip install pyLDAvis

In [ ]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [42]:
path = 'https://github.com/AsmaMora/Customer_Analytics/raw/main/Data/Got_to_grill_review.csv'

In [43]:
df = pd.read_csv(path)

In [45]:
df.tail()

,ReviewID,Review
5,6,ร้านน่านั่ง อาหารอร่อย ไม่ต้องรอนาน
6,7,อาหารญี่ปุ่น ปิ้งย่างฮาลาล ที่สุดใน กรุงเทพ
7,8,เนื้อนุ่ม พนักงานบริการช้านิดนึง แต่ก็โอเคค่ะ
8,9,บุฟเฟ่ต์ฮาลาล เนื้อย่างพรีเมียม สายเนื้อต้องไม...
9,10,เนื้อคือดีมาก


In [46]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['', '  ', '\n', 'ร้าน', '(', ')' ]
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [47]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [49]:
df.tail()

,ReviewID,Review,Review_tokenized
5,6,ร้านน่านั่ง อาหารอร่อย ไม่ต้องรอนาน,"นั่ง, ,อาหาร,อร่อย, ,ไม่ต้อง,รอ"
6,7,อาหารญี่ปุ่น ปิ้งย่างฮาลาล ที่สุดใน กรุงเทพ,"อาหาร,ญี่ปุ่น, ,ปิ้ง,ย่าง,ฮาลาล, , ,กรุงเทพ"
7,8,เนื้อนุ่ม พนักงานบริการช้านิดนึง แต่ก็โอเคค่ะ,"เนื้อ,นุ่ม, ,พนักงานบริการ,นิดนึง, ,โอเค"
8,9,บุฟเฟ่ต์ฮาลาล เนื้อย่างพรีเมียม สายเนื้อต้องไม...,"บุฟเฟ่ต์,ฮาลาล, ,เนื้อ,ย่าง,พรีเมียม, ,สาย,เนื..."
9,10,เนื้อคือดีมาก,"เนื้อ,ดีมาก"


In [ ]:
#Create Dictionary
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [ ]:
#Topic Modeling
num_topics = 4
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every =1 # Don't evaluate model perplexity, takes too much time

#make a index to word dictionary
temp = dictionary[0] #This is only to load the dictionary
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                                    alpha='auto', eta='auto', \
                                    iterations=iterations, num_topics=num_topics, \
                                    passes=passes, eval_every=eval_every)

In [ ]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

In [ ]:
model.show_topic(1)

In [ ]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [ ]:
df.tail()